In [1]:
!mkdir -p /scratch/sagarsj42

import os
os.chdir('/scratch/sagarsj42')
os.environ['TRANSFORMERS_CACHE'] = '/scratch/sagarsj42'

In [2]:
import torch
import librosa
import laion_clap
import numpy as np

/home2/sagarsj42/miniconda3/envs/video/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [3]:
!wget https://huggingface.co/lukewys/laion_clap/resolve/main/music_audioset_epoch_15_esc_90.14.pt

--2023-06-05 21:48:55--  https://huggingface.co/lukewys/laion_clap/resolve/main/music_audioset_epoch_15_esc_90.14.pt
Resolving huggingface.co (huggingface.co)... 108.158.46.32, 108.158.46.90, 108.158.46.5, ...
Connecting to huggingface.co (huggingface.co)|108.158.46.32|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/bd/48/bd4853aeff7fe8c507daac22cffef84fcbb89a75ded88f90cfcd47c280becc58/fae3e9c087f2909c28a09dc31c8dfcdacbc42ba44c70e972b58c1bd1caf6dedd?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27music_audioset_epoch_15_esc_90.14.pt%3B+filename%3D%22music_audioset_epoch_15_esc_90.14.pt%22%3B&Expires=1686241135&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2JkLzQ4L2JkNDg1M2FlZmY3ZmU4YzUwN2RhYWMyMmNmZmVmODRmY2JiODlhNzVkZWQ4OGY5MGNmY2Q0N2MyODBiZWNjNTgvZmFlM2U5YzA4N2YyOTA5YzI4YTA5ZGMzMWM4ZGZjZGFjYmM0MmJhNDRjNzBlOTcyYjU4YzFiZDFjYWY2ZGVkZD9yZXNwb25zZS1jb250ZW50L

In [4]:
model_path = 'music_audioset_epoch_15_esc_90.14.pt'

In [5]:
# For quantization
def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)

def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    return (x * 32767.).astype(np.int16)

In [6]:
model = laion_clap.CLAP_Module(enable_fusion=False, amodel='HTSAT-base')

model

/home2/sagarsj42/miniconda3/envs/video/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model fro

CLAP_Module(
  (model): CLAP(
    (audio_branch): HTSAT_Swin_Transformer(
      (spectrogram_extractor): Spectrogram(
        (stft): STFT(
          (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(480,), bias=False)
          (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(480,), bias=False)
        )
      )
      (logmel_extractor): LogmelFilterBank()
      (spec_augmenter): SpecAugmentation(
        (time_dropper): DropStripes()
        (freq_dropper): DropStripes()
      )
      (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (patch_embed): PatchEmbed(
        (proj): Conv2d(1, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          dim=128, input_resolution=(64, 64), depth=2
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
   

In [7]:
model.load_ckpt(model_path)

model

Load the specified checkpoint music_audioset_epoch_15_esc_90.14.pt from users.
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2

CLAP_Module(
  (model): CLAP(
    (audio_branch): HTSAT_Swin_Transformer(
      (spectrogram_extractor): Spectrogram(
        (stft): STFT(
          (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(480,), bias=False)
          (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(480,), bias=False)
        )
      )
      (logmel_extractor): LogmelFilterBank()
      (spec_augmenter): SpecAugmentation(
        (time_dropper): DropStripes()
        (freq_dropper): DropStripes()
      )
      (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (patch_embed): PatchEmbed(
        (proj): Conv2d(1, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          dim=128, input_resolution=(64, 64), depth=2
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
   

In [8]:
audio_file = [
    '/home2/sagarsj42/yt8m/data/sample.wav'
]

In [9]:
audio_emb = model.get_audio_embedding_from_filelist(x=audio_file, use_tensor=False)

type(audio_emb), audio_emb.shape

(numpy.ndarray, (1, 512))

In [10]:
audio_emb[:, -20:]

array([[-0.05279794, -0.05380071, -0.08961916, -0.00964758, -0.0385212 ,
        -0.00467488,  0.05111531, -0.02370409, -0.01228538, -0.0250142 ,
        -0.04035574, -0.04503557, -0.00160808, -0.02086216,  0.00052674,
        -0.00241546,  0.0735217 ,  0.05885008, -0.00421781,  0.00917581]],
      dtype=float32)

In [11]:
audio_emb = model.get_audio_embedding_from_filelist(x=audio_file, use_tensor=True)

type(audio_emb), audio_emb.shape

(torch.Tensor, torch.Size([1, 512]))

In [12]:
audio_emb.requires_grad

True

In [13]:
audio_data, _ = librosa.load(audio_file[0], sr=48000)

type(audio_data), audio_data.shape, audio_data[-20:]

(numpy.ndarray,
 (1609422,),
 array([-0.03420503, -0.030711  , -0.02740682, -0.02452059, -0.02223679,
        -0.02066216, -0.0198038 , -0.01956397, -0.01975277, -0.02011684,
        -0.02037902, -0.02028223, -0.01962992, -0.01831628, -0.0163413 ,
        -0.0138084 , -0.01090577, -0.00787481, -0.00497164, -0.00242835],
       dtype=float32))

In [14]:
audio_data = audio_data.reshape(1, -1)

audio_data.shape, audio_data[0, -20:]

((1, 1609422),
 array([-0.03420503, -0.030711  , -0.02740682, -0.02452059, -0.02223679,
        -0.02066216, -0.0198038 , -0.01956397, -0.01975277, -0.02011684,
        -0.02037902, -0.02028223, -0.01962992, -0.01831628, -0.0163413 ,
        -0.0138084 , -0.01090577, -0.00787481, -0.00497164, -0.00242835],
       dtype=float32))

In [1]:
audio_data_q = torch.from_numpy(int16_to_float32(float32_to_int16(audio_data))).float()

audio_data_q.dtype, audio_data_q[0, -20:]

NameError: name 'torch' is not defined

In [26]:
audio_embed = model.get_audio_embedding_from_data(audio_data_q, use_tensor=True)

audio_embed.shape, audio_embed[0, -20:], audio_embed.requires_grad

(torch.Size([1, 512]),
 tensor([-0.0666, -0.0302, -0.0902, -0.0276, -0.0396, -0.0089,  0.0281, -0.0264,
         -0.0277, -0.0249, -0.0295, -0.0435,  0.0135, -0.0239,  0.0038, -0.0061,
          0.0741,  0.0464, -0.0107,  0.0182], device='cuda:0',
        grad_fn=<SliceBackward0>),
 True)

In [27]:
text_data = ['Some sample music piece.', 'dog whistle']
text_embed = model.get_text_embedding(text_data, use_tensor=True)

type(text_embed), text_embed.shape, text_embed[:, -10:], text_embed.requires_grad

(torch.Tensor,
 torch.Size([2, 512]),
 tensor([[-0.1269, -0.0228,  0.0306,  0.0336, -0.0480, -0.0390,  0.0276, -0.0502,
           0.0521,  0.0108],
         [-0.0386, -0.0094, -0.0006,  0.0287,  0.0411, -0.0116,  0.0490, -0.0078,
          -0.0297,  0.1238]], device='cuda:0', grad_fn=<SliceBackward0>),
 True)